# JPEG decoder benchmark

- 보통 vision task 에서 data loader 가 bottleneck 일 때, jpeg decoder 가 보틀넥이 되는 경우가 많음.
  - 아니면 resize.
- NVIDIA DALI 를 쓸것이 아니라면, jpeg decoder 를 최대한 빠른 애로 쓰는 게 좋겠다.

In [1]:
import os
import time
from pathlib import Path
from PIL import Image
import cv2
import numpy as np
from turbojpeg import TurboJPEG

In [2]:
root = Path("/nfs/public/jbcha/kag/ranzcr/data/train/")
paths = list(root.glob("*.jpg"))

In [3]:
def benchmark(loader, paths, N=100):
    st = time.time()
    for path in paths[:N]:
        p = str(path)
        image = loader(p)
    ed = time.time()
    elapsed = ed - st
    
    print(f"Elapsed: {elapsed:.1f}s")

In [4]:
def sanity_check(loader_dic, path):
    images = []
    for name, loader in loader_dic.items():
        img = loader(str(path))
        images.append(img)
        print(name, img.shape, img.dtype)
    
    sanity = True
    loader_names = list(loader_dic.keys())
    for i, img1 in enumerate(images):
        for j, img2 in enumerate(images[i+1:]):
            if not (img1 == img2).all():
                n1 = loader_names[i]
                n2 = loader_names[j]
                print(f"{n1} != {n2}")
                sanity = False
    
    if sanity:
        print("Sanity check passed !!!")

In [5]:
def opencv_loader(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def pil_loader(path):
    image = Image.open(path).convert("RGB")
    return np.asarray(image)

turbojpeg = TurboJPEG()
def turbojpeg_loader(path):
    with open(path, "rb") as f:
        image = turbojpeg.decode(f.read())
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    return image

In [6]:
sanity_check({
    "opencv": opencv_loader,
    "pil": pil_loader,
    "turbo": turbojpeg_loader
}, paths[0])

opencv (2532, 3056, 3) uint8
pil (2532, 3056, 3) uint8
turbo (2532, 3056, 3) uint8
Sanity check passed !!!


In [7]:
sanity_check({
    "opencv": opencv_loader,
    "pil": pil_loader,
    "turbo": turbojpeg_loader
}, paths[1])

opencv (2650, 2790, 3) uint8
pil (2650, 2790, 3) uint8
turbo (2650, 2790, 3) uint8
Sanity check passed !!!


In [8]:
benchmark(opencv_loader, paths, N=500)

Elapsed: 26.9s


In [9]:
benchmark(pil_loader, paths, N=500)

Elapsed: 25.2s


In [10]:
benchmark(turbojpeg_loader, paths, N=500)

Elapsed: 15.8s
